## Problem 1-4 : NorthWinddatset https://relational.fit.cvut.cz/dataset/Northwind

How to download the dataset

To download the dataset you need to create connection in your mysql workbench

The datasets are publicly available directly from MariaDB database.

* Open your favourite MariaDB client (MySQL Workbench)
Use following credentials:
```
    hostname: relational.fit.cvut.cz
    port: 3306
    username: guest
    password: relational
```
Drive link: https://drive.google.com/drive/folders/1FMi6g1KDHbn17ikJSi7bE9-THywlxhdu?usp=sharing

###`Q-1`: Rank Employee in terms of revenue generation. Show employee id, first name, revenue, and rank


###`Q-2`: Show All products cumulative sum of units sold each month.


###`Q-3`: Show Percentage of total revenue by each suppliers



###`Q-4:` Show Percentage of total orders by each suppliers



### `Q-5`:Show All Products Year Wise report of totalQuantity sold, percentage change from last year.





In [ ]:
1.  SELECT e.EmployeeID, e.FirstName,
        SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) AS revenue,
        RANK() OVER(ORDER BY SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) DESC) AS revenue_rank
    FROM employees e
        JOIN orders o ON e.EmployeeID = o.EmployeeID
        JOIN order_details od ON od.OrderID = o.OrderID
    GROUP BY e.EmployeeID, e.FirstName
    ORDER BY revenue DESC;

2.  SELECT p.ProductName, YEAR(o.OrderDate) AS order_year,
		-- MONTH(o.OrderDate) AS order_month,
        MONTHNAME(o.OrderDate) AS order_month,
        SUM(od.Quantity) AS unit_sold,
        SUM(SUM(od.Quantity)) OVER(PARTITION BY p.ProductName ORDER BY
        YEAR(o.OrderDate), MONTH(o.OrderDate)) AS cumulative_unit_sold
    FROM products p
        JOIN order_details od ON p.ProductID = od.ProductID
        JOIN orders o ON o.OrderID = od.OrderID
    GROUP BY p.ProductName, YEAR(o.OrderDate), MONTH(o.OrderDate), MONTHNAME(o.OrderDate)
    ORDER BY p.ProductName;

3.  SELECT s.CompanyName AS supplier_name,
        SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) AS total_revenue,
        (SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) /
    (SELECT SUM(UnitPrice * Quantity * (1 - Discount)) FROM order_details)) * 100 AS percentage_of_total_revenue
    FROM suppliers s
        JOIN products p ON p.SupplierID = s.SupplierID
        JOIN order_details od ON od.ProductID = p.ProductID
    GROUP BY s.CompanyName
    ORDER BY percentage_of_total_revenue DESC;

4.  SELECT s.CompanyName AS supplier_name,
        COUNT(DISTINCT o.OrderID) AS total_orders,
        (COUNT(DISTINCT o.OrderID) / (SELECT COUNT(*) FROM orders)) * 100 AS percentage_of_total_orders
    FROM suppliers s
        JOIN products p ON p.SupplierID = s.SupplierID
        JOIN order_details od ON od.ProductID = p.ProductID
        JOIN orders o ON o.OrderID = od.OrderID
    GROUP BY s.CompanyName
    ORDER BY percentage_of_total_orders DESC;

5.  SELECT p.ProductName,
        YEAR(o.OrderDate) AS order_year,
        SUM(od.Quantity) AS total_quantity,
        LAG(SUM(od.Quantity), 1, 0) OVER(PARTITION BY p.ProductName ORDER BY YEAR(o.OrderDate)) AS prev_year_quantity,
        ((SUM(od.Quantity) - LAG(SUM(od.Quantity), 1, 0) OVER(PARTITION BY p.ProductName ORDER BY YEAR(o.OrderDate))) /
        LAG(SUM(od.Quantity), 1, 0) OVER (PARTITION BY p.ProductName ORDER BY YEAR(o.OrderDate))) * 100 AS percentage_change
    FROM products p
        JOIN order_details od ON od.ProductID = p.ProductID
        JOIN orders o ON o.OrderID = od.OrderID
    GROUP BY p.ProductName, YEAR(o.OrderDate)
    ORDER BY p.ProductName, order_year;

## Problem 6 - 15

For the next problems, you can get the dataset as well as the details [here](https://www.kaggle.com/datasets/thedevastator/drug-performance-evaluation?select=Drug_clean.csv) (Use the *cleaned* csv file).

### **`Problem-6:`** For each condition, what is the average satisfaction level of drugs that are "On Label" vs "Off Label"?


### **`Problem-7:`** For each drug type (RX, OTC, RX/OTC), what is the average ease of use and satisfaction level of drugs with a price above the median for their type?


### **`Problem 8:`** What is the cumulative distribution of EaseOfUse ratings for each drug type (RX, OTC, RX/OTC)? Show the results in descending order by drug type and cumulative distribution. (Use the built-in method and the manual method by calculating on your own. For the manual method, use the "ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW" and see if you get the same results as the built-in method.)


### **`Problem 9:`** What is the median satisfaction level for each medical condition? Show the results in descending order by median satisfaction level. (Don't repeat the same rows of your result.)



### **`Problem 10:`** What is the running average of the price of drugs for each medical condition? Show the results in ascending order by medical condition and drug name.



### **`Problem 11:`** What is the percentage change in the number of reviews for each drug between the previous row and the current row? Show the results in descending order by percentage change.



### **`Problem 12:`** What is the percentage of total satisfaction level for each drug type (RX, OTC, RX/OTC)? Show the results in descending order by drug type and percentage of total satisfaction.



### **`Problem 13:`** What is the cumulative sum of effective ratings for each medical condition and drug form combination? Show the results in ascending order by medical condition, drug form and the name of the drug.



### **`Problem-14:`** What is the rank of the average ease of use for each drug type (RX, OTC, RX/OTC)? Show the results in descending order by rank and drug type.



### **`Problem-15:`** For each condition, what is the average effectiveness of the top 3 most reviewed drugs?

In [ ]:
6.  SELECT `Condition`, Indication, AVG(Satisfaction) AS avg_satisfaction
    FROM drug
    WHERE Indication IN('On Label', 'Off Label')
    GROUP BY `Condition`, Indication
    ORDER BY `Condition`, Indication;

7.

8.  SELECT Type,EaseOfUse, CUME_DIST() OVER (PARTITION BY Type ORDER BY EaseOfUse DESC) AS CumulativeDistribution
    FROM drug
    ORDER BY Type DESC, CumulativeDistribution DESC;

    -- Manual Method:
    SELECT Type, EaseOfUse,
        SUM(1) OVER (PARTITION BY Type ORDER BY EaseOfUse DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) /
        COUNT(*) OVER (PARTITION BY Type) AS CumulativeDistribution
    FROM drug
    ORDER BY Type DESC, CumulativeDistribution DESC;

9.  SELECT DISTINCT `Condition`, AVG(Satisfaction) AS median_satisfaction
    FROM (
    SELECT `Condition`, Satisfaction,
        @cur_rank := IF(@prev_condition = `Condition`, @cur_rank + 1, 1) AS `rank`,
        @prev_condition := `Condition`
    FROM
        drug, (SELECT @cur_rank := 0, @prev_condition := NULL) r
    ORDER BY `Condition`, Satisfaction
    ) AS subquery
    WHERE `rank` = (SELECT FLOOR((COUNT(*) + 1) / 2) FROM drug WHERE `Condition` = subquery.`Condition`)
    GROUP BY `Condition`
    ORDER BY median_satisfaction DESC;

10. SELECT `Condition`,Drug, Price,
        AVG(Price) OVER (PARTITION BY `Condition` ORDER BY Drug ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS RunningAverage
    FROM drug
    ORDER BY `Condition` ASC, Drug ASC;

11. SELECT Drug, Reviews,
        COALESCE(LAG(Reviews) OVER (PARTITION BY Drug ORDER BY Reviews), 0) AS PreviousReviews,
    CASE
        WHEN COALESCE(LAG(Reviews) OVER (PARTITION BY Drug ORDER BY Reviews), 0) = 0 THEN 0
        ELSE ((Reviews - COALESCE(LAG(Reviews) OVER (PARTITION BY Drug ORDER BY Reviews), 0)) / COALESCE(LAG(Reviews) OVER (PARTITION BY Drug ORDER BY Reviews), 0)) * 100
    END AS PercentageChange
    FROM drug
    ORDER BY PercentageChange DESC;

12. SELECT  Type, SUM(Satisfaction) AS TotalSatisfaction,
        (SUM(Satisfaction) / SUM(SUM(Satisfaction)) OVER ()) * 100 AS PercentageOfTotalSatisfaction
    FROM drug
    GROUP BY Type
    ORDER BY Type DESC, PercentageOfTotalSatisfaction DESC;

13. SELECT `Condition`, Form, Drug, Effective,
        SUM(Effective) OVER (PARTITION BY `Condition`, Form ORDER BY Drug) AS CumulativeSum
    FROM drug
    ORDER BY `Condition` ASC, Form ASC, Drug ASC;

14. SELECT Type,AVG(EaseOfUse) AS AverageEaseOfUse,
    DENSE_RANK() OVER (ORDER BY AVG(EaseOfUse) DESC) AS `rank`
    FROM
    drug
    GROUP BY Type
    ORDER BY `rank` ASC, Type DESC;

15. SELECT `Condition`, AVG(Effective) AS AverageEffectiveness
    FROM (
    SELECT `Condition`, Drug, Effective, Reviews,
        DENSE_RANK() OVER (PARTITION BY `Condition` ORDER BY Reviews DESC) AS `rank`
    FROM drug
    ) AS subquery
    WHERE `rank` <= 3
    GROUP BY `Condition`
    ORDER BY `Condition` ASC;